## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/124-googleq-a-train-3fold-bert-base-unc-head-tail/' + '*.h5')
model_path_list.sort()
print('Models to predict:', model_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/124-googleq-a-train-3fold-bert-base-unc-head-tail/model_fold_1.h5', '/kaggle/input/124-googleq-a-train-3fold-bert-base-unc-head-tail/model_fold_2.h5', '/kaggle/input/124-googleq-a-train-3fold-bert-base-unc-head-tail/model_fold_3.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# Get beginning and end of long sentences
test['question_title'] = test['question_title'].apply(lambda x: x if len(x.split()) <= 30 else ' '.join(x.split()[:15] + x.split()[:-15]))
test['question_body'] = test['question_body'].apply(lambda x: x if len(x.split()) <= 239 else ' '.join(x.split()[:120] + x.split()[:-119]))
test['answer'] = test['answer'].apply(lambda x: x if len(x.split()) <= 239 else ' '.join(x.split()[:120] + x.split()[:-119]))

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.945264,0.580143,0.154302,0.470857,0.679951,0.610887,0.688240,0.627314,0.595909,...,0.857599,0.896376,0.567836,0.948873,0.930494,0.812664,0.076208,0.051357,0.854490,0.888771
1,46,0.889255,0.534705,0.010040,0.737490,0.779121,0.917648,0.558191,0.436287,0.087899,...,0.661289,0.939594,0.637901,0.961636,0.967202,0.840196,0.892550,0.106591,0.091271,0.847342
2,70,0.922617,0.662988,0.031577,0.768121,0.883208,0.874197,0.635930,0.490693,0.152924,...,0.854977,0.917419,0.584800,0.958674,0.958484,0.851827,0.090245,0.063997,0.858201,0.888693
3,132,0.862033,0.418630,0.009115,0.636316,0.763814,0.886483,0.583820,0.445297,0.155332,...,0.669204,0.934711,0.638476,0.962807,0.972136,0.844744,0.841707,0.142548,0.430273,0.878363
4,200,0.913916,0.406652,0.053598,0.759258,0.766902,0.804918,0.661895,0.554015,0.097432,...,0.669757,0.885715,0.612701,0.944774,0.946610,0.807830,0.157577,0.115836,0.657535,0.883550


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.884096,0.586254,0.036923,0.690746,0.806182,0.837958,0.592452,0.476061,0.250763,...,0.771724,0.920624,0.641275,0.954152,0.961587,0.840366,0.507333,0.135134,0.518872,0.885868
std,2812.670060,0.051367,0.136377,0.059754,0.110898,0.095181,0.126271,0.053829,0.086988,0.192424,...,0.093425,0.026384,0.049234,0.013930,0.013375,0.041072,0.322258,0.066867,0.279288,0.025528
min,39.000000,0.724582,0.290678,0.003545,0.200744,0.345994,0.252275,0.478868,0.325712,0.010521,...,0.544927,0.784696,0.535230,0.890392,0.894945,0.723776,0.006811,0.006024,0.030061,0.780147
25%,2572.000000,0.850142,0.467821,0.010364,0.623594,0.759152,0.804449,0.552302,0.413987,0.105728,...,0.696031,0.906710,0.607364,0.945592,0.955275,0.811955,0.167175,0.083344,0.267090,0.869844
50%,5093.000000,0.889418,0.584641,0.015252,0.691553,0.810009,0.882814,0.584201,0.447530,0.188042,...,0.779238,0.924746,0.635541,0.956102,0.964520,0.843254,0.583931,0.137151,0.524018,0.888248
75%,7482.000000,0.923055,0.696991,0.030686,0.764133,0.870062,0.918217,0.627872,0.523707,0.354336,...,0.854834,0.938826,0.670201,0.964068,0.970863,0.870205,0.809579,0.185156,0.759636,0.905227
max,9640.000000,0.971801,0.880479,0.580347,0.954444,0.979057,0.964247,0.751897,0.769019,0.814265,...,0.937773,0.970198,0.797981,0.981382,0.986259,0.942295,0.955009,0.338668,0.986641,0.940242
